In [1]:
import pymysql
from flask import Flask, request, jsonify, session
from flask import redirect, url_for, send_from_directory, render_template
from werkzeug.utils import secure_filename
import json
from datetime import datetime, date
import datetime as dt
from PIL import Image
import io
from io import BytesIO
from base64 import encodebytes
import time
%config Completer.use_jedi = False

import os
import cv2
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

In [2]:
from tensorflow.keras.models import load_model
model = load_model('./ps_model/datasets/my_model_acc_91.h5')

In [3]:
def centroid_histogram(clt):
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)
    hist = hist.astype("float")
    hist /= hist.sum()
    return hist

def plot_colors(hist, centroids):
    bar = np.zeros((50, 300, 3), dtype="uint8")
    startX = 0
    size_lst = []
    for (percent, color) in zip(hist, centroids):
        endX = startX + (percent * 300)
        cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
                      color.astype("uint8").tolist(), -1)
        size_lst.append(startX-endX)
        startX = endX
    rep_ind = size_lst.index((-1)*max(np.abs(np.array(size_lst))))
    rep_color = centroids[rep_ind]
    
    return bar, rep_color

In [4]:
# 이미지 채도 분석 모델
class imgmodel:
    def test(vidcap):
        path = './ps_model/result/'
        test = os.listdir(path)
        for item in test:
            if item.endswith(".jpeg"):
                os.remove(os.path.join(path, item))

        # 영상의 의미지를 연속적으로 캡쳐할 수 있게 하는 class
    #     vidcap = cv2.VideoCapture('./ps_model/datasets/healing.mp4') # 여기 영상 넣으셈
        count = 0
        # get() 함수를 이용하여 전체 프레임 중 1/20의 프레임만 가져와 저장
        ret=True
        while(ret):
            ret, image = vidcap.read()#return 값과 image를 읽어옴
            if(ret==False):
                break

            if(int(vidcap.get(1)) % 30 == 0):
                cv2.imwrite("./ps_model/result/frame%d.jpeg" % count, image)
                count += 1
        vidcap.release()

        img_array = []
        image_path = r'./ps_model/result'
        for img in os.listdir(image_path):
            pic = cv2.imread(os.path.join(image_path,img))
            img_array.append(pic)

        arr = []
        for i in range(len(img_array)):
            arr = cv2.cvtColor(img_array[i], cv2.COLOR_BGR2RGB)
        for i in range(len(arr)):
            image = arr.reshape((arr.shape[0] * arr.shape[1], 3))
        k = 5
        clt = KMeans(n_clusters = k)
        clt.fit(image)
        hist = centroid_histogram(clt)
        bar,rep_color = plot_colors(hist, clt.cluster_centers_)
        res = pd.DataFrame(rep_color)
        result = res.T
        pre = model.predict(result)
        r = pre.argmax(axis=1)
        img_tag = pd.read_csv('./ps_model/datasets/img_tag3.csv', encoding='UTF-8')
        result_2 = img_tag.iloc[r[0]+1]['tag']
        
        return result_2


In [6]:
# 실제로 우리가 파라미터로 이미지 경로를 입력할 함수입니다.

def detect_img(image_url):
    start_time = time.time()
    result = run_detector(detector, image_url)
    end_time = time.time()
    return result          
    
# 실제로 딥러닝을 돌리는 구간입니다. 
def run_detector(detector, path):
    img = load_img(path) 
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    start_time = time.time()
    result = detector(converted_img)
    end_time = time.time()
    result = {key:value.numpy() for key,value in result.items()}      
    entities = {} 
    for entity in result['detection_class_entities'] : 
    
        entity = str(entity)[2:-1]
    
        if entity in entities : 
            entities[entity] = entities[entity] + 1 
        else : 
            entities[entity] = 0 
        
    return(sorted(entities.items(), key=lambda x: x[1], reverse=True))
  
  #이미지를 보여주기전에 객체를 탐지한 부분을 박스 처리해 보여주는 구간입니다. 
  #우리는 박스처리한 이미지를 보여줄 필요 없이 태그만 뽑아오는게 목적이니까 
  #테스트를 하고 나서 이 부분을 쓸지 말지 고민해보면 좋을것 같습니다. 
    image_with_boxes = draw_boxes(
        img.numpy(), result["detection_boxes"],
        result["detection_class_entities"], result["detection_scores"])

    display_image(image_with_boxes)



# 이미지를 처리할 수 있도록 읽어와서 디코드합니다. 
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img


In [7]:
# 객체 분석 모델 사용
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [8]:
class objectmodel:
    def test(root_dir):
        root_dir = './ps_model/result/' # 디렉토리

        img_path_list = []
        possible_img_extension = ['.jpeg'] # 이미지 확장자들

        for (root, dirs, files) in os.walk(root_dir):
            if len(files) > 0:
                for file_name in files:
                    if os.path.splitext(file_name)[1] in possible_img_extension:
                        img_path = root + '/' + file_name

                    # 경로에서 \를 모두 /로 바꿔줘야함
                        img_path = img_path.replace('\\', '/') # \는 \\로 나타내야함         
                        img_path_list.append(img_path)
        aa = []
        img_arr2 = []
        for i in range(len(img_path_list)):
            aa.append(img_path_list[i])
            img_arr2.append(aa[i].split('//'))
            del img_arr2[i][0]
        img_arr3=sum(img_arr2,[])

        dir_path = './ps_model/result/'   # 현재 경로를 알려주는 함수입니다. 
        image_url2 = []
        object_arr = []
        for i in range(len(img_arr3)):
            file_name = f'./frame{i}.jpeg' # 내가 실제로 입력할 파일명 
            image_url2.append(os.path.join(dir_path , file_name))
            object_result=detect_img(image_url2[i])
            object_arr.append(object_result[:10])
            
#         return object_arr

        count_dic = {}
        for li in object_arr:
            for item in li:
                if item[0] not in count_dic.keys():
                    count_dic[item[0]] = item[1]
                else:
                    count_dic[item[0]] = item[1] + count_dic[item[0]]
        tmp = [x[0] for x in sorted(count_dic.items(), key=lambda x: x[1], reverse=True)[:2]]
        object_tag = pd.read_csv('./ps_model/datasets/object_tag.csv')
        df=object_tag[object_tag['object'].str.contains(tmp[0])]
        a=df.values.tolist()
        result_tag= a[0][1]
        result2 = []
        result2.append(result_tag)
        object_return = result2+tmp
        return object_return

In [ ]:
from flask import Flask
from werkzeug.serving import WSGIRequestHandler
WSGIRequestHandler.protocol_version = "HTTP/1.1"

db = pymysql.connect(host='project-db-stu.ddns.net',
                        port=3307,
                        user='www3mu',
                        password='12345',
                        db='www3mu',
                        charset='utf8')
cur = db.cursor()
root_dir = './ps_model/result/'
# 이미지 전송 함수
def get_respones_image(image_path):
    pil_img = Image.open(image_path, mode='r') # 이미지 불러옴
    byte_arr = io.BytesIO()
    pil_img.save(byte_arr, format='jpeg')
    encoded_img = encodebytes(byte_arr.getvalue()).decode('ascii') # encode as base64
    return encoded_img

app = Flask(__name__)

@app.route('/upload', methods = ['GET', 'POST']) # 영상업로드 체크
def upload_file():
    if request.method == 'POST':
        f = request.files['file']
        # 저장할 경로 + 파일명
        f.save('mp4/'+f.filename)
        return 'uploads 디렉토리 -> 파일 업로드 성공!'
    
@app.route('/music/upload', methods = ['GET', 'POST']) # 음악업로드 체크
def music_upload():
#     if request.method == 'POST': 
        
        music = request.files['music'] # 음악 저장
        cover = request.files['cover'] # 커버사진 저장
        genre = request.form['genre'] # 음악 장르
        music_name = request.form['name'] # 음악 이름
        tag1 = request.form['tag1'] # 태그
        tag2 = request.form['tag2']
        tag3 = request.form['tag3']
        Login_id = request.form['id'] # 로그인 정보

        # 저장할 경로 + 파일명
        cover.save('music/image/'+cover.filename)
        music.save('music/'+music.filename)
        
        music_load = "C:/Users/smhrd/PS프로젝트/music/"+music.filename
        cover_load = "C:/Users/smhrd/PS프로젝트/music/image/"+cover.filename
        
        sql = f"insert into tbl_music (mu_name, m_id, mu_genre, mu_cover, mu_tag1, mu_tag2, mu_tag3, mu_file)values ('{music_name}', '{Login_id}', '{genre}', '{cover_load}', '{tag1}','{tag2}', '{tag3}', '{music_load}')"
                
        cur.execute(sql)
        momo = cur.fetchone()
        db.commit()  
        
        return 'music 디렉토리 -> 파일 업로드 성공!'
    
# 로그인 체크    
@app.route('/LoginSelect', methods = ['GET', 'POST'])
def LoginChick():
    if request.method == 'POST':
                
        Login_id = request.form['id']
        Login_pw = request.form['pw']
        Login_type = request.form['type']
        
        sql = f"select * from tbl_member where m_id = '{Login_id}' and m_pw = '{Login_pw}'"
                
        cur.execute(sql)
        momo = cur.fetchone()
        db.commit()        
#         db.close()
        
        a = 1 # 로그인 실패
        b = 2 # 로그인 성공 ( 크리에이터 1 )
        c = 3 # 로그인 성공 ( 아티스트 0 )
        print(momo)
        
        if None == momo:
            return str(1)
        elif Login_id == momo[0] and Login_pw == momo[1] and momo[4] == 'a':
            return str(3)
        elif Login_id == momo[0] and Login_pw == momo[1] and momo[4] == 'c':
            return str(2)
        
# 이미지 보내기    
@app.route('/image', methods = ['GET', 'POST'])
def imageupload():
#     if request.method == 'POST':
                
        Login_id = request.form['id']
        sql = f"select m_image from tbl_member where m_id = '{Login_id}'"
        cur.execute(sql)
        path = cur.fetchall()
#         print(get_respones_image(path))
        
        encoded_imges = []
        for image_path in path:
            encoded_imges.append(get_respones_image(image_path[0]))
#         print(encoded_imges)
            
        db.commit()        
#         db.close()
        
        return jsonify({"result":encoded_imges})
    
# 회원가입
@app.route('/MBJoin', methods = ['GET', 'POST'])
def joinChick():
    if request.method == 'POST':
        
        time = datetime.now()
        PSid = request.form['id']
        PSpw = request.form['pw']
        PSnick = request.form['pwc']
        PSname = request.form['name']
        PSdate = request.form['date']
        PSsex = request.form['sex']
        PSurl = request.form['url']
        PStype = request.form['type']
        PSimgFile = request.files['imgFile']
#         PSimgFile.save(secure_filename(PSimgFile.filename))
        PSimgFile.save("image/"+PSimgFile.filename)
        img_load = "C:/Users/smhrd/PS프로젝트/image/"+PSimgFile.filename
        print(PSid, PSpw, PSnick, PSname, PSsex, PSdate, PSurl, PSimgFile)
        print(img_load)
        
        sql = f"insert into tbl_member (m_id, m_pw, m_nick, m_gender, m_type, m_joindate, m_url, m_image) values ('{PSid}', '{PSpw}', '{PSnick}', '{PSsex}', '{PStype}','{time}', '{PSurl}', '{img_load}')"

        cur.execute(sql)
        db.commit()
#         db.close()
        
        return "회원가입 정보확인!"

# 아티스트 - 내음악
@app.route('/artistList', methods = ['GET', 'POST'])
def artistList():
#     if request.method == 'POST':
        cur = db.cursor(pymysql.cursors.DictCursor)
                
        Login_id = request.form['id']
        sql = f"select * from tbl_music where m_id = '{Login_id}'"
        cur.execute(sql)
        path = cur.fetchall()
#         print(get_respones_image(path))
        
        encoded_imges = []
        for image_path in path:
            encoded_imges.append(get_respones_image(image_path["mu_cover"]))

        print(path)
        db.commit()        
        
        return jsonify({"from":path, "img":encoded_imges})
    
    
# 추천 리스트 ->> 링크
@app.route('/Rink', methods = ['GET', 'POST'])
def Rink():
#     if request.method == 'POST':
        cur = db.cursor(pymysql.cursors.DictCursor)
                
        art = request.form['art']
        sql = f"select * from tbl_member where m_id = '{art}'"
        cur.execute(sql)
        path = cur.fetchall()
#         print(get_respones_image(path))
        
        
        db.commit()        
        
        return jsonify({'art':path})

    
# 크리에이터 - 추천 리스트
@app.route('/ModelList', methods = ['GET', 'POST'])
def modelList():
        cur = db.cursor(pymysql.cursors.DictCursor)
        vidcap = cv2.VideoCapture('./ps_model/datasets/test2.mp4') # 경로
        tag1 = imgmodel.test(vidcap) # 1번 결과값, 태그    
        nowtime = dt.datetime.now()
        nowtime_1 = datetime.strftime(nowtime,"%Y_%m_%d_%H_%M_%S" ) # 파일명을 넣기위함
        nowtimefl = nowtime_1+"_file.mp4"

        taglist = objectmodel.test(root_dir)
        tag2 = taglist[0] # 2번 결과값, 태그
        tag3 = taglist[1] # 3번 태그
        tag4 = taglist[2] # 4번 태그
        
        sql = f'''select * from tbl_music
                where (mu_tag2 IN ('{tag1}' , '{tag2}') or  mu_tag3 in ('{tag1}' , '{tag2}'))
                or (mu_tag2 IN ('{tag1}' , '{tag2}') and  mu_tag1 in ('{tag1}' , '{tag2}' )
                or mu_tag1 IN ('{tag1}' , '{tag2}') and  mu_tag3 in ('{tag1}' , '{tag2}' ))'''
        
        cur.execute(sql)
        path = cur.fetchall()
        
        encoded_imges = []
        for image_path in path:
            encoded_imges.append(get_respones_image(image_path["mu_cover"]))
            
            
        db.commit()
        return_fl = []
        
#         print(jsonify({'from':path, 'img':encoded_imges}))

        return jsonify({'from':path, 'img':encoded_imges,'tag1':tag1, 'tag2':tag2,'tag4':tag4, 'tag3':tag3})

# 즐겨찾기 추가
@app.route('/favoriate', methods = ['GET', 'POST'])
def favoriate():
#     if request.method == 'POST':
        cur = db.cursor(pymysql.cursors.DictCursor)
                
        Login_id = request.form['id']
        fav_seq = request.form['seq']
        sql = f'''insert into www3mu.tbl_favoriate 
            (mu_seq, m_id, fav_date) values('{fav_seq}', '{Login_id}', '2020-02-22')'''

        cur.execute(sql)
#         print(get_respones_image(path))

        db.commit()        
        
        return '즐겨찾기 추가 성공'
    
# 즐겨찾기 추가
@app.route('/favoriateX', methods = ['GET', 'POST'])
def favoriateX():
#     if request.method == 'POST':
        cur = db.cursor(pymysql.cursors.DictCursor)
                
        Login_id = request.form['id']
        fav_seq = request.form['seq']
        sql = f'''delete from tbl_favoriate where mu_seq = {fav_seq}'''

        cur.execute(sql)
#         print(get_respones_image(path))

        db.commit()        
        
        return '즐겨찾기 삭제 성공'

# 즐겨찾기 리스트 출력부
@app.route('/favoriatelist', methods = ['GET', 'POST'])
def favoriatelist():
    
        cur = db.cursor(pymysql.cursors.DictCursor)
                
        Login_id = request.form['id']
        sql = f'''select * from tbl_music where mu_seq in (select mu_seq from tbl_favoriate where m_id = '{Login_id}');'''
        
        cur.execute(sql)
        path = cur.fetchall()
        
        encoded_imges = []
        for image_path in path:
            encoded_imges.append(get_respones_image(image_path["mu_cover"]))

            
#         filepath = []
#         for file in path:
#             filepath.append(send_file(file['mu_file']))
#             print(filepath)
        db.commit()        
        
        return jsonify({"from":path, "img":encoded_imges})


from flask import send_file
# 음악 선택 실행부분
@app.route('/Musicplay', methods = ['GET', 'POST'])
def Musicplay():
        cur = db.cursor()
              
        seq = request.form['seq']
        sql = f"select mu_file from tbl_music where mu_seq = '{seq}'"
        
        cur.execute(sql)
        momo = cur.fetchall()
        
        asdf1 = list(momo)[0]
        music = list(asdf1)[0]
                         
        db.commit() 
        
        return send_file(music, "audio/mp3")

# 랭킹 음악 선택 *실행* 부분
@app.route('/RankingMusicplay', methods = ['GET', 'POST'])
def RankingMusicplay():
        cur = db.cursor()
              
        name = request.form['name']
        sql = f"select mu_file from tbl_music where mu_name = '{name}'"
        
        cur.execute(sql)
        momo = cur.fetchall()
        
        asdf1 = list(momo)[0]
        music = list(asdf1)[0]
                         
        db.commit() 
        
        return send_file(music, "audio/mp3")

    
# 랭킹 음악 선택 *출력* 부분
@app.route('/RankingMusicList', methods = ['GET', 'POST'])
def RankingMusicList():
        cur = db.cursor(pymysql.cursors.DictCursor)
              
        sql = '''select * from tbl_music 
where mu_seq in ('113','127','7','12','149')
order by
CASE WHEN mu_seq = '127' THEN 1 
WHEN mu_seq = '113' THEN 2
WHEN mu_seq = '7' THEN 3 
WHEN mu_seq = '12' THEN 4
WHEN mu_seq = '149' THEN 5 
END ASC;'''
        
        cur.execute(sql)
        momo = cur.fetchall()
        
#         name = momo[1] # 제목
#         m_id = momo[2] # 가수
        encoded_imges = []
        for image_path in momo:
            encoded_imges.append(get_respones_image(image_path["mu_cover"]))
        
        db.commit() 
        
        return jsonify({'from':momo, 'img':encoded_imges})
app.run(host = '172.30.1.41', port=5000)
# http://172.30.1.41:5000/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.30.1.41:5000/ (Press CTRL+C to quit)
INFO:werkzeug:172.30.1.41 - - [27/Jan/2022 18:05:33] "POST /LoginSelect HTTP/1.1" 200 -


None


INFO:werkzeug:172.30.1.41 - - [27/Jan/2022 18:05:43] "POST /LoginSelect HTTP/1.1" 200 -


('MatFix', '1234', 'MatFix', 'm', 'a', datetime.datetime(2022, 1, 13, 0, 0), 'https://www.jamendo.com/artist/530223/matfix', None)


INFO:werkzeug:172.30.1.41 - - [27/Jan/2022 18:06:17] "POST /LoginSelect HTTP/1.1" 200 -


('MatFix', '1234', 'MatFix', 'm', 'a', datetime.datetime(2022, 1, 13, 0, 0), 'https://www.jamendo.com/artist/530223/matfix', None)


INFO:werkzeug:172.30.1.41 - - [27/Jan/2022 18:06:47] "POST /music/upload HTTP/1.1" 200 -
INFO:werkzeug:172.30.1.41 - - [27/Jan/2022 18:06:50] "POST /artistList HTTP/1.1" 200 -


[{'mu_seq': 54, 'mu_name': 'Fashion City', 'm_id': 'MatFix', 'mu_genre': '하우스', 'mu_cover': 'C:/Users/smhrd/PS프로젝트/music/image/Fashion City.jpg', 'mu_tag1': '센치함', 'mu_tag2': '신나는', 'mu_tag3': '모던한', 'mu_file': 'C:/Users/smhrd/PS프로젝트/music/Fashion City.mp3'}, {'mu_seq': 92, 'mu_name': 'MatFix - Faded', 'm_id': 'MatFix', 'mu_genre': '하우스', 'mu_cover': 'C:/Users/smhrd/PS프로젝트/music/image/MatFix - Faded.jpg', 'mu_tag1': '센치함', 'mu_tag2': '신나는', 'mu_tag3': '모던한', 'mu_file': 'C:/Users/smhrd/PS프로젝트/music/MatFix - Faded.mp3'}, {'mu_seq': 198, 'mu_name': '5gSanta', 'm_id': 'MatFix', 'mu_genre': '일렉트로닉', 'mu_cover': 'C:/Users/smhrd/PS프로젝트/music/image/2022-01-27-09-06-26_Cover.jpeg', 'mu_tag1': '평화로움', 'mu_tag2': '몽환적인', 'mu_tag3': '미래적인', 'mu_file': 'C:/Users/smhrd/PS프로젝트/music/2022-01-27-09-06-21_Muisc.mp3'}]


INFO:werkzeug:172.30.1.41 - - [27/Jan/2022 18:06:59] "POST /RankingMusicList HTTP/1.1" 200 -
